In [ ]:
from sklearn.datasets import load_iris
import torch
from torch.utils.data import DataLoader, TensorDataset, random_split
from torch import nn
from tqdm import tqdm
import matplotlib.pyplot as plt
import os

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

In [ ]:
iris = load_iris()

In [ ]:
x = torch.tensor(iris.data, dtype=torch.float32)
y = torch.tensor(iris.target, dtype=torch.long)

In [ ]:
dataset = TensorDataset(x, y)

In [ ]:
train_data , test_data = random_split(dataset, [0.8, 0.2])

In [ ]:
train_data = DataLoader(train_data, batch_size=16, shuffle=True)
test_data = DataLoader(test_data, batch_size=16, shuffle=False)

In [ ]:
for batch_X, batch_y in train_data:
    print(len(batch_X), len(batch_y))
    break

In [ ]:
class MyModel(nn.Module):
    def __init__(self, input, output):
        super().__init__()
        self.layer1 = nn.Linear(input, 16)
        self.act1 = nn.ReLU()
        self.layer2 = nn.Linear(16, 8)
        self.act2 = nn.ReLU()
        self.layer3 = nn.Linear(8, output)
    def forward(self, x):
        x = self.act1(self.layer1(x))
        x = self.act2(self.layer2(x))
        x = self.layer3(x)
        return x
model = MyModel(4, 3).to(device)

In [ ]:
loss = nn.CrossEntropyLoss()
opt = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(opt, mode='min', factor=0.1, patience=5)

In [ ]:
train_loss = []
train_acc = []
val_loss = []
val_acc = []
lr_list = []

In [ ]:
epochs = 1000

for epoch in range(epochs):
    if epoch % 50 == 0:
        os.system('cls' if os.name == 'nt' else 'clear')
    lr = scheduler._last_lr
    train_loop = tqdm(train_data, leave=False)
    true_answer = 0
    running_train_loss = []
    model.train()
    total_answers = 0
    for x, target in train_loop:
        x = x.to(device)
        target = target.to(device)
        pred = model(x)
        los = loss(pred, target)
        opt.zero_grad()
        los.backward()
        opt.step()
        running_train_loss.append(los.item())
        mean_train_loss = sum(running_train_loss) / len(running_train_loss)
        true_answer += (pred.argmax(dim=1) == target).sum().item()
        total_answers += target.size(0)
        train_loop.set_description(F"epoch:[{epoch+1}/{epochs}], train_loss:[{mean_train_loss}]")
    running_train_acc = true_answer/ total_answers
    train_loss.append(mean_train_loss)
    train_acc.append(running_train_acc)

    model.eval()
    with torch.no_grad():
        true_answer = 0
        running_val_loss = []
        total_answers = 0
        for x, target in test_data:
            x = x.to(device)
            target = target.to(device)
            pred = model(x)
            los = loss(pred, target)
            running_val_loss.append(los.item())
            mean_val_loss = sum(running_val_loss) / len(running_val_loss)
            true_answer += (pred.argmax(dim=1) == target).sum().item()
            total_answers += target.size(0)
        running_val_acc = true_answer/ total_answers
        val_loss.append(mean_val_loss)
        val_acc.append(running_val_acc)

    if epoch%50 == 0:
        print(f"epoch [{epoch+1}/{epochs}], train_loss = {mean_train_loss:.4f}, train_acc = {running_train_acc:.4f}, val_loss = {mean_val_loss:.4f}, val_acc = {running_val_acc:.4f}, lr = {lr}")
    scheduler.step(mean_val_loss)
    lr_list.append(lr)

In [ ]:
plt.plot(train_acc)
plt.plot(val_acc)
plt.legend(['train_acc', 'val_acc'])
plt.show()

In [ ]:
plt.plot(train_loss)
plt.plot(val_loss)
plt.legend(['train_loss', 'val_loss'])
plt.show()

In [ ]:
plt.plot(lr_list)
plt.legend(['lr'])
plt.show()

In [ ]:
torch.save(model.state_dict(),'model.pt')